# 0. 패키지 임포트

In [1]:
from dotenv import load_dotenv
from pinecone import Pinecone
from langchain_pinecone import PineconeVectorStore
from langchain_upstage import UpstageEmbeddings
from langchain_openai import OpenAIEmbeddings
from langchain import hub
from langchain_openai import ChatOpenAI
from langchain.chains import RetrievalQA
from decouple import config
import pandas as pd
import numpy as np
from sklearn.metrics.pairwise import cosine_similarity
from langchain.retrievers.multi_query import MultiQueryRetriever
from langchain.chains.query_constructor.base import AttributeInfo
from langchain.retrievers.self_query.base import SelfQueryRetriever
from langchain_cohere import CohereRerank
from langchain.retrievers import ContextualCompressionRetriever

c:\Users\Admin\anaconda3\envs\llm-lab\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


# 1. 기본 세팅

In [2]:
load_dotenv()

True

In [3]:
pc = Pinecone()

In [4]:
# embedding = OpenAIEmbeddings(model="text-embedding-3-large")
embedding = UpstageEmbeddings(model="solar-embedding-1-large")
index_name = config("PINECONE_INDEX")


In [5]:
database = PineconeVectorStore(
    embedding=embedding, 
    index_name=index_name
)

In [6]:
retriever = database.as_retriever(search_kwargs={"k": 5})
prompt = hub.pull('rlm/rag-prompt')
llm = ChatOpenAI(model="gpt-5-nano")

In [7]:
compressor = CohereRerank(model="rerank-multilingual-v3.0", top_n=3)

compression_retriever = ContextualCompressionRetriever(
    base_compressor=compressor, 
    base_retriever=retriever
)


In [8]:
metadata_field_info = [
    AttributeInfo(
        name="source",
        description="법률의 이름. 예시: 개인금융채권의 관리 및 개인금융채무자의 보호에 관한 법률 시행령(대통령령)(제34945호)(20241017).docx",
        type="string",
    ),
]
document_content_description = "대한민국 법률 조항"

self_query_retriever = SelfQueryRetriever.from_llm(
    llm=llm,
    vectorstore=database,
    document_contents=document_content_description,
    metadata_field_info=metadata_field_info,
    enable_limit=True,  
    search_kwargs={"k": 3},
)

In [9]:
retriever_from_llm = MultiQueryRetriever.from_llm(
    retriever=retriever, 
    llm=llm,
)

In [10]:
qa_chain = RetrievalQA.from_chain_type(
    llm,
    retriever = compression_retriever, # 리트리버 방식 변경 가능 (retriever, retriever_from_llm, self_query_retriever, compression_retriever )
    chain_type_kwargs={"prompt": prompt},
)

# 2. 선다형 문제로 검증 실행

In [11]:
df = pd.read_csv("객관식_문제.csv")
df.head(2)

,Unnamed: 0,문제유형,난이도,문제내용,보기1,보기2,보기3,보기4,정답,해설
0,개인금융채권의 관리 및 개인금융채무자의 보호에 관한 법률 시행령(대통령령)(제349...,사지선다형,상,「개인금융채권의 관리 및 개인금융채무자의 보호에 관한 법률 시행령(대통령령)(제34...,서면 또는 전자문서,전화 통화,구두 통지,우편 발송,1,제14조에 따르면 채권추심자는 서면 또는 전자문서로 통지를 해야 한다.
1,개인금융채권의 관리 및 개인금융채무자의 보호에 관한 법률 시행령(대통령령)(제349...,사지선다형,상,「개인금융채권의 관리 및 개인금융채무자의 보호에 관한 법률 시행령(대통령령)(제34...,채무자의 요청,법령,금융위원회 고시,채권자의 요구,2,제15조에 따르면 추심연락의 구체적인 기준은 법령에 따라 정해진다.


In [12]:
success, fail = 0, 0
fail_questions = []
gpt = []
act = []
for i in range(100):
    sample = df.loc[i]
    question = """아래 문제를 보고 답을 고르시오.
만약 보기 중 3번째 보기가 정답이라면 3 이라고 출력하시오.
보기가 A,B,C,D 라면 1,2,3,4 라고 생각하고 답변하시오.
오직 숫자만 출력하시오.
출력 예시: 3
"""
    question += sample['문제내용'] + '\n'

    for i in range(1,5):
        col = '보기' + str(i)
        question += sample[col] + '\n'

    message = qa_chain.invoke({"query":question})['result']
    if message == str(sample['정답']):
        success += 1
    else:
        fail += 1
        fail_questions.append(sample['문제내용']) # 틀린문제만 추출해서 살펴보기
    gpt.append(message)
    act.append( str(sample['정답']))

In [13]:
print('정답 갯수 :', success)
print('오답 갯수 :', fail)
print('정확도 :', (success)/(success+fail))
print('챗봇에 대답: ', gpt)
print('실제 답    : ', act)

정답 갯수 : 80
오답 갯수 : 20
정확도 : 0.8
챗봇에 대답:  ['1', '3', '3', '3', '2', '2', '4', '3', '3', '3', '2', '2', '3', '3', '2', '2', '3', '1', '3', '2', '2', '2', '모르겠습니다.', '3', '1', '4', '4', '2', '1', '1', '1', '2', '1', '1', '1', '4', '2', '1', '2', '1', '1', '1', '1', '1', '2', '2', '2', '1', '1', '1', '3', '3', '1', '1', '1', '4', '1', '1', '2', '2', '2', '1', '4', '2', '1', '1', '1', '2', '1', '1', '3', '3', '1', '2', '2', '1', '4', '1', '1', '3', '2', '3', '1', '3', '1', '1', '1', '2', '2', '4', '2', '2', '1', '2', '4', '1', '1', '3', '1', '1']
실제 답    :  ['1', '2', '3', '3', '2', '2', '4', '3', '3', '3', '2', '1', '2', '3', '1', '2', '3', '1', '3', '2', '2', '3', '1', '3', '1', '1', '4', '2', '1', '1', '1', '2', '2', '1', '1', '3', '1', '1', '2', '1', '1', '1', '1', '1', '2', '2', '2', '1', '1', '1', '3', '3', '2', '1', '1', '1', '1', '1', '2', '2', '2', '1', '1', '2', '1', '2', '1', '1', '1', '1', '1', '3', '1', '2', '2', '1', '4', '1', '1', '3', '2', '3', '1', '2', '1', '1', '1', '2', 

## 2-1. 틀린문제 분석

In [14]:
for q in fail_questions:
    document_data = database.similarity_search_with_score(q)[0][0]
    max_similarity = database.similarity_search_with_score(q)[0][1] # 해당 문제에 대한 소스 중 가장 높은 유사도 점수
    print('문제 내용 :', q)
    print('참고 문서: ',document_data.metadata)
    print('참고 내용:',document_data.page_content)
    print('최대 유사도 :', max_similarity)
    print()

문제 내용 : 「개인금융채권의 관리 및 개인금융채무자의 보호에 관한 법률 시행령(대통령령)(제34945호)(20241017)」 제15조에 따르면 추심연락의 구체적인 기준은 무엇에 따라 정해지는가?
참고 문서:  {'source': '개인금융채권의 관리 및 개인금융채무자의 보호에 관한 법률 시행령(대통령령)(제34945호)(20241017).docx'}
참고 내용: 개인금융채권의 관리 및 개인금융채무자의 보호에 관한 법률 시행령

개인금융채권의 관리 및 개인금융채무자의 보호에 관한 법률 시행령 ( 약칭: 개인채무자보호법 시행령 )

[시행 2024. 10. 17.] [대통령령 제34945호, 2024. 10. 16., 제정]

금융위원회(서민금융과) 02-2100-2612



제1장 총칙
최대 유사도 : 0.713896811

문제 내용 : 「개인금융채권의 관리 및 개인금융채무자의 보호에 관한 법률 시행령(대통령령)(제34945호)(20241017)」 제14조에 따른 개인금융채권 추심의 착수 통지 방법은 무엇인가?
참고 문서:  {'source': '개인금융채권의 관리 및 개인금융채무자의 보호에 관한 법률(법률)(제20369호)(20241017).docx'}
참고 내용: 제15조(추심의 착수 통지) 채권추심자는 개인금융채권의 추심에 착수하려는 경우(채권금융회사등이 제2조제1호가목의 행위를 원인으로 보유하게 된 개인금융채권의 추심에 직접 착수하려는 경우는 제외한다)에는 대통령령으로 정하는 바에 따라 제2호의 추심 착수 예정일의 3영업일 전까지 다음 각 호의 사항을 개인금융채무자에게 통지하여야 한다.

1. 채권자, 채무금액, 연체금액 및 연체기간 등 추심 대상 개인금융채권에 관한 사항

2. 추심 착수 예정일

3. 채권추심자의 성명ㆍ명칭 및 연락처(채권추심자가 법인인 경우에는 추심업무를 담당하는 사람의 성명 및 연락처를 포함한다)

4. 연락ㆍ독촉 등 수행하려는 추심업무의 방법

5. 불법추심이나 소멸시효 완성채권의 추심에 대한 대응요령 등 대통령령으로 정하는

# 3. 단답형 문제로 검증 실행

In [15]:
df = pd.read_csv("단답형_문제.csv")
df.sample()

,법령명,문제유형,난이도,문제내용,정답,해설
22,금융거래지표의 관리에 관한 법률 시행령(대통령령)(제31193호)(20201127),단답형,상,금융거래지표의 관리에 관한 법률 시행령의 약칭은 무엇인가?,금융거래지표법 시행령,법령 본문에 명시된 약칭이다.


In [16]:
results = []
SIMILARITY_THRESHOLD = 0.7

In [17]:
success, fail = 0, 0
gpt = []
act = []
for i in range(50):

    question = """아래 문제를 보고 짧고 간단하게 답을 단답식으로 제출하시오.
    좋은 답변들 예시 : ['대주주와의 거래', '지배구조법 대주주', '금융위원회', '제12조', '5%']
    이와같이 30자 이내로 핵심 단어의 단답형으로 답변하시오.
    """

    sample = df.loc[i]
    question += sample['문제내용']
    answer = sample['정답']


    
    message = qa_chain.invoke({"query": question})['result']

    ground_truth_embedding = embedding.embed_query(answer)
    prediction_embedding = embedding.embed_query(message)
    
    score = cosine_similarity(
        np.array(ground_truth_embedding).reshape(1, -1),
        np.array(prediction_embedding).reshape(1, -1)
    )[0][0]
    
    if score >= SIMILARITY_THRESHOLD:
        success += 1
    else:
        fail += 1
        
    gpt.append(message)
    act.append(answer)

In [18]:
print('정답 갯수 :', success)
print('오답 갯수 :', fail)
print('정확도 :', (success)/(success+fail))
print('챗봇에 대답: ', gpt)
print('실제 답    : ', act)

정답 갯수 : 25
오답 갯수 : 25
정확도 : 0.5
챗봇에 대답:  ['모른다', '모르겠습니다. 주어진 맥락에 해당 사유의 목록이 제시되지 않았습니다.', '모르겠습니다.', '자기자본의 1만분의 10 또는 10억원 중 적은 금액', '대통령령으로 정하는 바에 따라 5영업일 전 통지', '모른다', '금융채권자 중 금융업무·기업구조조정 업무를 수행하는 기관', '상환기일 연장, 원리금 감면, 채권의 출자전환', '채무 이행 청구 행위', '10영업일 전', '개인금융채권을 추심하는 자', '제9조', '제10조', '제1조의2', '정부등', '당해 국제금융기구', '2024년 12월 3일', '정보 없음', '예산에 반영', '한국은행 총재', '제16580호', '금융위원회', '금융거래지표법 시행령', '정관', '5명', '대체지표 안내', '2020.11.24', '법 제6조제4항', '금융거래지표법', '시장거래에 기반한 수치·호가·평가액·추정치 등의 자료', '2019년', '산출업무규정 제정', '매년 7월 31일까지', '자기자본의 1만분의 10과 10억원 중 적은 금액', '건전한경영과금융시장안정도모및금융소비자보호', '금융업을 영위하는 회사', '2021년 6월 30일', '금융복합기업집단법', '요건 미충족', '대주주와의 내부거래', '지배구조법 제2조제6호 대주주', '금융위원회', '제12조', '5%', '주요출자자', '대주주 신용공여 한도', '주요출자자', '대주주 신용공여 한도 보고', '대통령령으로 정하는 금융업 영위 회사', '재무구조 악화로 영업 지속 어려운 금융기관']
실제 답    :  ['법 제3조제1항 각 호의 권리로 담보된 개인금융채권', '연체기간이나 연체횟수를 초과한 경우', '3개월', '5천만원', '서면 또는 전자문서', '채권금융회사가 보유한 개인금융채무자에 대한 금전채권', '개인금융채권을 보유한 자', '채무의 내용을 변경하는 것', '채권의 만족을 얻기 위한 모든 행위', '10영업일 전